In [1]:
import requests
import csv

# switch platform id = 7m6ylw9p
# switch virtual console platform id = mx6p1463

# Spelunky Classic = 76rp9q18
# Platforms
# PC= 8gej2n93

# Spelunky HD = 2685ok6p
# Platforms
# Playstation 3 = mx6pwe3g
# XBox 360 = n568oevp
# Xbox One X = 4p9z70er
# Playstation 4 = nzelkr6q
# PC = 8gej2n93

# Spelunky 2 = 369p0r31
# Platforms
# Playstation 4 = nzelkr6q
# PC = 8gej2n93

def platformName(platformID):
    if platformID == '8gej2n93':
        return 'PC'
    elif platformID == 'mx6pwe3g':
        return 'Playstation 3'
    elif platformID == 'nzelkr6q':
        return 'Playstation 4'
    elif platformID == 'n568oevp':
        return 'XBox 360'
    elif platformID == '4p9z70er':
        return 'XBox One'
    else:
        return 'Unknown'

gameIDs = ['76rp9q18', '2685ok6p', '369p0r31']
urlBase = 'https://www.speedrun.com/api/v1/runs?game='
gameNameURLBase = 'https://www.speedrun.com/api/v1/games/'
columnNames = [ "Run ID", "User ID", "User Name", "Hours", "Minutes", "Seconds", "Platform ID", "Platform Name", "Verified", "URL", "Date"]

for x in range(0,len(gameIDs)):
    url = urlBase + gameIDs[x]
    gameURL = gameNameURLBase + gameIDs[x]
    gameNameR = requests.get(gameURL, timeout=(3.05,27))
    gameNameJSON = gameNameR.json()
    gameName = gameNameJSON['data']['names']['international']
    fileName = gameName.replace(" ", "") + ".csv"

    
    with open(fileName, 'w', newline='') as myfile:
        wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
        wr.writerow(columnNames)
    
        i = 20
        r = requests.get(url, timeout=(3.05, 27))
        ourJSON = r.json()
        while ourJSON['data'] != []:
            # do our data pull
            for y in range(0, len(ourJSON['data'])):
                toWrite = []
                toWrite.append(ourJSON['data'][y]['id'])
                userID = ourJSON['data'][y]['players'][0]['id']
                toWrite.append(userID)
                userURL = ourJSON['data'][y]['players'][0]['uri']
                p = requests.get(userURL, timeout=(3.05,27))
                userJSON = p.json()
                toWrite.append(userJSON['data']['names']['international'])

                # time

                time = ourJSON['data'][y]['times']['primary']
                # removes PT from playtime
                time = time[2:]
                hour, h, remainder = time.partition('H')
                if hour.isdigit():
                    toWrite.append(float(hour))
                    time = remainder
                else:
                    toWrite.append(0)
                    time = hour
                minute, m, remainder = time.partition('M')
                if minute.isdigit():
                    toWrite.append(float(minute))
                    time = remainder
                else:
                    toWrite.append(0)
                    time = minute
                remainTime, s ,remainder = time.partition('S')
                second, dot, millis = remainTime.partition('.')
                if second.isdigit():
                    if millis.isdigit():
                        toWrite.append(float(second)+(float(millis)/(10**(len(millis)))))
                    else:
                        toWrite.append(float(second))
                else:
                    toWrite.append(0)

                # platform

                platformID = ourJSON['data'][y]['system']['platform']
                toWrite.append(platformID)
                toWrite.append(platformName(platformID))

                toWrite.append(ourJSON['data'][y]['status']['status'])

                toWrite.append(ourJSON['data'][y]['links'][0]['uri'])

                toWrite.append(ourJSON['data'][y]['date'])

                wr.writerow(toWrite)
                    

            # get next page
            newURL = url + '&offset=' + str(i)
            r = requests.get(newURL, timeout=(3.05,27))
            ourJSON = r.json()
            i = i + 20
            #ourJSON['data'] = [] 
        

#with open('runs.csv', 'w', newline='') as myfile:
#    wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
#    wr.writerow(mylist2)
